In [1]:
!pip install tensorflow==2.15

In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.15.0'

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle

In [4]:
!kaggle datasets download -d saumyamohandas/animal-dataset


Dataset URL: https://www.kaggle.com/datasets/saumyamohandas/animal-dataset
License(s): unknown
 98% 61.0M/62.3M [00:03<00:00, 24.0MB/s]
100% 62.3M/62.3M [00:03<00:00, 17.4MB/s]


In [5]:
!unzip '/content/animal-dataset.zip'

Archive:  /content/animal-dataset.zip
  inflating: dataset/Testing/bears/k4 (100).jpeg  
  inflating: dataset/Testing/bears/k4 (100).jpg  
  inflating: dataset/Testing/bears/k4 (101).jpeg  
  inflating: dataset/Testing/bears/k4 (101).jpg  
  inflating: dataset/Testing/bears/k4 (102).jpeg  
  inflating: dataset/Testing/bears/k4 (102).jpg  
  inflating: dataset/Testing/bears/k4 (103).jpeg  
  inflating: dataset/Testing/bears/k4 (104).jpeg  
  inflating: dataset/Testing/bears/k4 (105).jpeg  
  inflating: dataset/Testing/bears/k4 (106).jpeg  
  inflating: dataset/Testing/bears/k4 (107).jpeg  
  inflating: dataset/Testing/bears/k4 (108).jpeg  
  inflating: dataset/Testing/bears/k4 (109).jpeg  
  inflating: dataset/Testing/bears/k4 (110).jpeg  
  inflating: dataset/Testing/bears/k4 (71).jpg  
  inflating: dataset/Testing/bears/k4 (72).jpeg  
  inflating: dataset/Testing/bears/k4 (72).jpg  
  inflating: dataset/Testing/bears/k4 (73).jpeg  
  inflating: dataset/Testing/bears/k4 (73).jpg  
  in

In [6]:
trainPath = '/content/dataset/Training'
testPath = '/content/dataset/Testing'

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,zoom_range=0.2,shear_range=0.2)
test_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
train = train_datagen.flow_from_directory(trainPath,target_size=(224,224),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(224,224),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


# VGG16

In [10]:
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [11]:
vgg = VGG16(include_top=False,input_shape=(224,224,3))

58889256/58889256 [==============================] - 4s 0us/step


In [12]:
vgg.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [13]:
for layer in vgg.layers:
  print(layer)

In [14]:
for layer in vgg.layers:
  layer.trainable=False

In [15]:
x = Flatten()(vgg.output)

In [16]:
output = Dense(4,activation='softmax')(x)

In [17]:
vgg16 = Model(vgg.input,output)

In [18]:
vgg16.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [19]:
vgg16.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [20]:
vgg16.fit(train,validation_data=test,epochs=2)

Epoch 1/2
78/78 [==============================] - 27s 266ms/step - loss: 0.5874 - accuracy: 0.7827 - val_loss: 0.0707 - val_accuracy: 1.0000
Epoch 2/2
78/78 [==============================] - 19s 243ms/step - loss: 0.1035 - accuracy: 0.9717 - val_loss: 0.0312 - val_accuracy: 1.0000


In [21]:
vgg16.save('Vgg.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [22]:
from keras.models import load_model

In [23]:
model1=load_model('/content/Vgg.h5')

In [24]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

In [25]:
index=['bears','crows','elephant','rats']

In [31]:
Img_path= r'/content/dataset/Testing/elephants/Z (19).jpeg'
img=image.load_img(Img_path,target_size=(224,224))
x=image.img_to_array(img)
x=np.expand_dims(x,axis=0) #(1,224,224,3)
img_data=preprocess_input(x)
img_data.shape
output=np.argmax(model1.predict(img_data), axis=1)
result = index[output[0]]
result

1/1 [==============================] - 0s 19ms/step


'elephant'

# Resnet

In [32]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [33]:
resnet50 = ResNet50(include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 4s 0us/step


In [34]:
for layer in resnet50.layers:
  print(layer)

In [35]:
for layer in resnet50.layers:
  layer.trainable=False

In [36]:
x = Flatten()(resnet50.output)

In [37]:
output = Dense(4,activation='softmax')(x)

In [38]:
resnet50 = Model(resnet50.input,output)

In [39]:
resnet50.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_2[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 conv1_bn (BatchNormalizati  (None, 112, 112, 64)         256       ['conv1_conv[0][0]']          
 on)                                                                                        

In [40]:
resnet50.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [41]:
resnet50.fit(train,validation_data=test,epochs=5)

Epoch 1/5
78/78 [==============================] - 24s 244ms/step - loss: 2.8394 - accuracy: 0.3675 - val_loss: 1.3462 - val_accuracy: 0.4448
Epoch 2/5
78/78 [==============================] - 19s 249ms/step - loss: 1.3434 - accuracy: 0.5081 - val_loss: 0.8346 - val_accuracy: 0.6779
Epoch 3/5
78/78 [==============================] - 20s 258ms/step - loss: 1.1112 - accuracy: 0.5905 - val_loss: 0.8146 - val_accuracy: 0.6871
Epoch 4/5
78/78 [==============================] - 19s 240ms/step - loss: 1.2928 - accuracy: 0.5557 - val_loss: 0.8174 - val_accuracy: 0.6564
Epoch 5/5
78/78 [==============================] - 18s 225ms/step - loss: 0.9992 - accuracy: 0.6454 - val_loss: 1.0492 - val_accuracy: 0.5798


# Inception

In [ ]:
#!pip install tensorflow==2.13

In [42]:
train = train_datagen.flow_from_directory(trainPath,target_size=(299,299),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(299,299),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [43]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [44]:
InceptionV3 = InceptionV3(include_top=False,input_shape=(299 ,299,3))

87910968/87910968 [==============================] - 5s 0us/step


In [45]:

for layer in InceptionV3.layers:
  print(layer)

In [46]:
x = Flatten()(InceptionV3.output)

In [47]:
output = Dense(4,activation='softmax')(x)

In [48]:
InceptionV3 = Model(InceptionV3.input,output)

In [49]:
InceptionV3.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 149, 149, 32)         864       ['input_3[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 149, 149, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 149, 149, 32)         0         ['batch_normalization[0]

In [50]:
InceptionV3.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [51]:
InceptionV3.fit(train,validation_data=test,epochs=5)

Epoch 1/5
78/78 [==============================] - 80s 511ms/step - loss: 1.7616 - accuracy: 0.7431 - val_loss: 7556.7866 - val_accuracy: 0.2239
Epoch 2/5
78/78 [==============================] - 32s 413ms/step - loss: 1.9334 - accuracy: 0.6947 - val_loss: 2910.5156 - val_accuracy: 0.2178
Epoch 3/5
78/78 [==============================] - 34s 429ms/step - loss: 2.0449 - accuracy: 0.5792 - val_loss: 704380.9375 - val_accuracy: 0.2607
Epoch 4/5
78/78 [==============================] - 33s 428ms/step - loss: 1.7261 - accuracy: 0.5630 - val_loss: 30772.1289 - val_accuracy: 0.2607
Epoch 5/5
78/78 [==============================] - 33s 427ms/step - loss: 1.8661 - accuracy: 0.6220 - val_loss: 605458.4375 - val_accuracy: 0.2607


# Xception

In [52]:
train = train_datagen.flow_from_directory(trainPath,target_size=(299,299),batch_size=16)
test = test_datagen.flow_from_directory(testPath,target_size=(299,299),batch_size=16)

Found 1238 images belonging to 4 classes.
Found 326 images belonging to 4 classes.


In [53]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.models import Model

In [54]:
Xception = Xception(include_top=False,input_shape=(299,299,3))

83683744/83683744 [==============================] - 5s 0us/step


In [55]:
x = Flatten()(Xception.output)

In [56]:
output = Dense(4,activation='softmax')(x)

In [57]:
Xception = Model(Xception.input,output)

In [58]:
Xception.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 299, 299, 3)]        0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, 149, 149, 32)         864       ['input_4[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, 149, 149, 32)         128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, 149, 149, 32)         0         ['block1_conv1_bn[0][0]'

In [59]:
Xception.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [60]:
Xception.fit(train,validation_data=test,epochs=2)

Epoch 1/2
78/78 [==============================] - 78s 597ms/step - loss: 0.8845 - accuracy: 0.8481 - val_loss: 10.9173 - val_accuracy: 0.4908
Epoch 2/2
78/78 [==============================] - 44s 555ms/step - loss: 0.3140 - accuracy: 0.9095 - val_loss: 2.3075 - val_accuracy: 0.8374
